In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [84]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

train.head(3)
train.tail(3)

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
4749,4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4750,4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4751,4751,19.1,987.6,1.0,0.3,0.0,1006.8,987.8,21.2,1007.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0


In [80]:
"""
train['date'] = pd.PeriodIndex(start='2020-01-01', end=None, periods=len(train), freq='10Min')
test['date'] = pd.PeriodIndex(start='2020-02-03', end=None, periods=len(test), freq='10Min')
train.head(15)

train['minute'] = (train['id']%6)*10
train.head(15)
"""

"\ntrain['date'] = pd.PeriodIndex(start='2020-01-01', end=None, periods=len(train), freq='10Min')\ntest['date'] = pd.PeriodIndex(start='2020-02-03', end=None, periods=len(test), freq='10Min')\ntrain.head(15)\n"

In [85]:
train['hour'] = (train['id']//6) - (train['id']//144)*24
test['hour'] = (test['id']//6) - (test['id']//144)*24

In [86]:
train = pd.get_dummies(train, columns=['hour'], prefix='H_')
test = pd.get_dummies(test, columns=['hour'], prefix='H_')
train.head(15)

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,H__14,H__15,H__16,H__17,H__18,H__19,H__20,H__21,H__22,H__23
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,...,0,0,0,0,0,0,0,0,0,0
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,...,0,0,0,0,0,0,0,0,0,0
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,...,0,0,0,0,0,0,0,0,0,0
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,...,0,0,0,0,0,0,0,0,0,0
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,...,0,0,0,0,0,0,0,0,0,0
5,5,9.1,988.9,1.3,0.7,0.0,1009.1,989.6,12.0,1010.0,...,0,0,0,0,0,0,0,0,0,0
6,6,9.1,988.8,2.4,1.9,0.0,1009.1,989.6,12.0,1010.1,...,0,0,0,0,0,0,0,0,0,0
7,7,8.9,988.9,1.0,1.2,0.0,1009.2,989.7,11.9,1010.1,...,0,0,0,0,0,0,0,0,0,0
8,8,9.0,988.9,1.5,0.1,0.0,1009.2,989.6,11.9,1010.0,...,0,0,0,0,0,0,0,0,0,0
9,9,8.8,988.9,1.4,0.4,0.0,1009.4,989.6,11.8,1010.1,...,0,0,0,0,0,0,0,0,0,0


In [87]:
def same_min_max(df):
    return df.drop(df.columns[df.max() == df.min()], axis=1, inplace=True)

In [88]:
same_min_max(train)
same_min_max(test)

In [90]:
# 평균기온
mean_temp = train.loc[:,"Y00":"Y18"].mean(axis=1)

In [91]:
t = train["Y18"].isna() 
null_index = t[t==True].index
# 평균기온으로 Y18 매움
train.loc[null_index, "Y18"] = train.loc[null_index, "Y00":"Y18"].mean(axis=1)

In [97]:
# Drop
train.drop(columns=['X05','X08','X09','X23','X33','Y03', 'Y04'], axis=1, inplace=True)
test.drop(columns=['X05','X08','X09','X23','X33','Y03', 'Y04'], axis=1, inplace=True)


KeyError: "['X05' 'X08' 'X09' 'X23' 'X33' 'Y04'] not found in axis"

In [99]:
train.columns

Index(['id', 'X00', 'X01', 'X02', 'X03', 'X04', 'X06', 'X07', 'X10', 'X11',
       'X12', 'X13', 'X15', 'X17', 'X18', 'X20', 'X21', 'X22', 'X24', 'X25',
       'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X34', 'X35', 'X36',
       'X37', 'X38', 'X39', 'Y00', 'Y01', 'Y02', 'Y05', 'Y06', 'Y07', 'Y08',
       'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17', 'Y18',
       'H__0', 'H__1', 'H__2', 'H__3', 'H__4', 'H__5', 'H__6', 'H__7', 'H__8',
       'H__9', 'H__10', 'H__11', 'H__12', 'H__13', 'H__14', 'H__15', 'H__16',
       'H__17', 'H__18', 'H__19', 'H__20', 'H__21', 'H__22', 'H__23'],
      dtype='object')

In [100]:
def standardization(df):
    mean = np.mean(df)
    std = np.std(df)
    norm = (df - mean) / (std - 1e-07)
    return norm, mean, std

In [101]:
X_name = train.loc[:,"X00":"X39"].columns
X_name

Index(['X00', 'X01', 'X02', 'X03', 'X04', 'X06', 'X07', 'X10', 'X11', 'X12',
       'X13', 'X15', 'X17', 'X18', 'X20', 'X21', 'X22', 'X24', 'X25', 'X26',
       'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X34', 'X35', 'X36', 'X37',
       'X38', 'X39'],
      dtype='object')

In [102]:
train_X_norm, mean, std = standardization(train[X_name]) 

#테스트용 데이터 표준화
test_X_norm = (test[X_name] - mean) / (std - 1e-07)

In [103]:
train_X_norm.shape, train.shape

((4752, 32), (4752, 74))

In [115]:
train.columns

Index(['id', 'X00', 'X01', 'X02', 'X03', 'X04', 'X06', 'X07', 'X10', 'X11',
       'X12', 'X13', 'X15', 'X17', 'X18', 'X20', 'X21', 'X22', 'X24', 'X25',
       'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X34', 'X35', 'X36',
       'X37', 'X38', 'X39', 'Y00', 'Y01', 'Y02', 'Y05', 'Y06', 'Y07', 'Y08',
       'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17', 'Y18',
       'H__0', 'H__1', 'H__2', 'H__3', 'H__4', 'H__5', 'H__6', 'H__7', 'H__8',
       'H__9', 'H__10', 'H__11', 'H__12', 'H__13', 'H__14', 'H__15', 'H__16',
       'H__17', 'H__18', 'H__19', 'H__20', 'H__21', 'H__22', 'H__23'],
      dtype='object')

In [110]:
train[["H__0","H__23"]]

,H__0,H__23
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0
6,0,0
7,0,0
8,0,0
9,0,0


In [117]:
train2 = pd.concat([train[["H__0","H__1","H__2","H__3","H__4","H__5","H__6","H__7","H__8","H__9","H__10","H__11","H__12","H__13","H__14","H__15","H__16","H__17","H__18","H__19","H__20","H__21","H__22","H__23"]], train_X_norm], axis=1) 
test2 = pd.concat([test[["H__0","H__1","H__2","H__3","H__4","H__5","H__6","H__7","H__8","H__9","H__10","H__11","H__12","H__13","H__14","H__15","H__16","H__17","H__18","H__19","H__20","H__21","H__22","H__23"]], test_X_norm], axis=1) 

In [ ]:
#train2 = pd.concat([train['date'], train_X_norm], axis=1) 
#test2  = pd.concat([test['date'] , test_X_norm] , axis=1)

print(train2.shape, test2.shape)

In [ ]:
train2.head()

In [ ]:
test2.head()

In [118]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
train['Y18']

In [119]:
lgb_train = lgb.Dataset(train2, label=train['Y18'])

lgb_param = {
    "objective":"regression",
    "metrics":"mae",
    "learning_rate":0.01
}

print("cv start")
cv_result = lgb.cv(
    lgb_param,
    lgb_train,
    num_boost_round=99999,
    nfold=5,
    early_stopping_rounds=10,
    stratified=False,
    verbose_eval=10 
)

print("train start")
lgb_model = lgb.train(
    lgb_param,
    lgb_train,
    num_boost_round=len(cv_result["l1-mean"])
)

cv start
[10]	cv_agg's l1: 5.57447 + 0.0878679
[20]	cv_agg's l1: 5.05873 + 0.0815209
[30]	cv_agg's l1: 4.59149 + 0.0752476
[40]	cv_agg's l1: 4.16935 + 0.0694612
[50]	cv_agg's l1: 3.78826 + 0.0643729
[60]	cv_agg's l1: 3.44404 + 0.0605311
[70]	cv_agg's l1: 3.13389 + 0.056258
[80]	cv_agg's l1: 2.85476 + 0.0521398
[90]	cv_agg's l1: 2.60431 + 0.0485568
[100]	cv_agg's l1: 2.37906 + 0.0454061
[110]	cv_agg's l1: 2.17778 + 0.0427429
[120]	cv_agg's l1: 1.99756 + 0.0395259
[130]	cv_agg's l1: 1.8355 + 0.0366561
[140]	cv_agg's l1: 1.69067 + 0.0343711
[150]	cv_agg's l1: 1.56086 + 0.0319445
[160]	cv_agg's l1: 1.44538 + 0.0299767
[170]	cv_agg's l1: 1.3416 + 0.0280855
[180]	cv_agg's l1: 1.24898 + 0.0272566
[190]	cv_agg's l1: 1.1658 + 0.0267182
[200]	cv_agg's l1: 1.0919 + 0.0258042
[210]	cv_agg's l1: 1.0259 + 0.024984
[220]	cv_agg's l1: 0.966578 + 0.0247578
[230]	cv_agg's l1: 0.914053 + 0.0241943
[240]	cv_agg's l1: 0.868004 + 0.02372
[250]	cv_agg's l1: 0.827579 + 0.0233763
[260]	cv_agg's l1: 0.791175 + 

[2050]	cv_agg's l1: 0.374234 + 0.0141288
[2060]	cv_agg's l1: 0.374006 + 0.0141422
[2070]	cv_agg's l1: 0.373825 + 0.0141711
[2080]	cv_agg's l1: 0.37365 + 0.0142294
[2090]	cv_agg's l1: 0.373433 + 0.014245
[2100]	cv_agg's l1: 0.373258 + 0.0142322
[2110]	cv_agg's l1: 0.373099 + 0.0142553
[2120]	cv_agg's l1: 0.372929 + 0.014255
[2130]	cv_agg's l1: 0.372749 + 0.0142746
[2140]	cv_agg's l1: 0.372605 + 0.0143096
[2150]	cv_agg's l1: 0.372434 + 0.0143327
[2160]	cv_agg's l1: 0.372267 + 0.0143029
[2170]	cv_agg's l1: 0.372086 + 0.014287
[2180]	cv_agg's l1: 0.371895 + 0.0143107
[2190]	cv_agg's l1: 0.371721 + 0.0143447
[2200]	cv_agg's l1: 0.371572 + 0.0143606
[2210]	cv_agg's l1: 0.371408 + 0.014362
[2220]	cv_agg's l1: 0.371254 + 0.0143713
[2230]	cv_agg's l1: 0.371107 + 0.0143502
[2240]	cv_agg's l1: 0.370936 + 0.0143461
[2250]	cv_agg's l1: 0.370758 + 0.014329
[2260]	cv_agg's l1: 0.370578 + 0.0143522
[2270]	cv_agg's l1: 0.370432 + 0.0143117
[2280]	cv_agg's l1: 0.370259 + 0.0143058
[2290]	cv_agg's l1: 0.

[4060]	cv_agg's l1: 0.356115 + 0.0136244
[4070]	cv_agg's l1: 0.35609 + 0.0136223
[4080]	cv_agg's l1: 0.356047 + 0.0136256
[4090]	cv_agg's l1: 0.356018 + 0.0136286
[4100]	cv_agg's l1: 0.355978 + 0.0136323
[4110]	cv_agg's l1: 0.355925 + 0.0136318
[4120]	cv_agg's l1: 0.355897 + 0.0136151
[4130]	cv_agg's l1: 0.35586 + 0.0136153
[4140]	cv_agg's l1: 0.355828 + 0.0136132
[4150]	cv_agg's l1: 0.355783 + 0.0136248
[4160]	cv_agg's l1: 0.355758 + 0.0136163
[4170]	cv_agg's l1: 0.355724 + 0.0136038
[4180]	cv_agg's l1: 0.355686 + 0.0136061
[4190]	cv_agg's l1: 0.355657 + 0.0136111
[4200]	cv_agg's l1: 0.35563 + 0.0136118
[4210]	cv_agg's l1: 0.355607 + 0.0136157
[4220]	cv_agg's l1: 0.355578 + 0.0136138
[4230]	cv_agg's l1: 0.355545 + 0.0136088
[4240]	cv_agg's l1: 0.35551 + 0.0136024
[4250]	cv_agg's l1: 0.355473 + 0.0135914
[4260]	cv_agg's l1: 0.355437 + 0.013588
[4270]	cv_agg's l1: 0.355397 + 0.0135872
[4280]	cv_agg's l1: 0.355352 + 0.0135989
[4290]	cv_agg's l1: 0.355303 + 0.0136081
[4300]	cv_agg's l1: 0

In [ ]:
#test2.drop(['date'], axis=1, inplace=True)
test2.head()

In [120]:
#submission["Y18"] = lgb_model.predict(test.loc[:, "X00":"X39"])
submission["Y18"] = lgb_model.predict(test2)
submission.head()

,id,Y18
0,4752,20.215582
1,4753,20.150096
2,4754,19.979501
3,4755,19.745174
4,4756,19.740227


In [121]:
submission.to_csv('data/submission_0406_1.csv', index=False)